In [1]:
from utils.read_file import *

In [2]:
QUESTION_PATH = "競賽資料集/dataset/preliminary/questions_example.json"
ANS_PATH = "競賽資料集/dataset/preliminary/ground_truths_example.json"
FINANCE_PATH = "競賽資料集/reference/finance"

In [ ]:
question = read_json(QUESTION_PATH)["questions"]

In [ ]:
from rank_bm25 import BM25Okapi

def get_ans_bm25(question_data: dict, documents: list, k=1):
    
    tokenized_documents = [doc.split(" ") for doc in documents]

    # bm25 模型
    bm25 = BM25Okapi(tokenized_documents)

    user_query = question_data["query"].split(" ")

    scores = bm25.get_scores(user_query)
    
    k_highest = np.argsort(scores)[-k:][::-1]
    return k_highest, np.sort(scores)[::-1][:k]


In [6]:
import numpy as np
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [14]:
def validate_finance_bm25(question):
    ans = read_json(ANS_PATH)
    correct = 0
    half_correct = 0

    for i in range(50, 100):
        src = question[i]["source"]
        document, order_list = read_target_insurance_pdf(FINANCE_PATH, src)
        
        # 直接使用完整的 document
        token_index, similarities = get_ans_bm25(question[i], document, 5)
        all_predict = [order_list[idx] for idx in token_index]
        predict = all_predict[0]

        if predict == ans["ground_truths"][i]["retrieve"]:
            correct += 1
        else:
            print(f"qid: {i + 1}, predict: {predict}, ans: {ans['ground_truths'][i]['retrieve']}")
            error_text = f"qid: {i + 1}, ans: {ans['ground_truths'][i]['retrieve']}\n\n\n"
            for j in range(len(token_index)):
                error_text += f"index: {order_list[token_index[j]]}, similarity: {similarities[j]}\n\n{document[token_index[j]]}\n\n\n"

            with open(f"output_finance/qid_{i + 1}.txt", 'w', encoding='utf-8') as file:
                file.write(error_text)

        if ans["ground_truths"][i]["retrieve"] in all_predict:
            half_correct += 1

    print(f"acc: {correct / 50 * 100} %")
    print(f"in rank 5: {half_correct / 50 * 100} %")


In [15]:
import os
if not os.path.exists("output_finance/"):
    os.makedirs("output_finance/")

validate_finance_bm25(question)

qid: 53, predict: 611, ans: 351
qid: 54, predict: 663, ans: 612
qid: 55, predict: 116, ans: 166
qid: 59, predict: 190, ans: 632
qid: 61, predict: 59, ans: 900
qid: 64, predict: 150, ans: 124
qid: 68, predict: 955, ans: 942
qid: 70, predict: 495, ans: 490
qid: 71, predict: 873, ans: 920
qid: 72, predict: 92, ans: 204
qid: 79, predict: 947, ans: 65
qid: 80, predict: 80, ans: 213
qid: 82, predict: 33, ans: 831
qid: 92, predict: 715, ans: 55
qid: 94, predict: 291, ans: 699
qid: 97, predict: 279, ans: 282
qid: 98, predict: 835, ans: 692
acc: 66.0 %
in rank 5: 100.0 %
